In [1]:
from qsharp import init, code, circuit, TargetProfile, BitFlipNoise
import qsharp.openqasm as qasm
from qsharp_widgets import Circuit, Histogram

In [2]:
init(target_profile=TargetProfile.Base)

# Read the QASM source file into a string and load it
with open("4e4o_ethylene_2det.qasm", "r") as file:
    qasm_src = file.read()
qasm.import_openqasm(qasm_src, name="ethylene")

circ_data = circuit(code.ethylene)
Circuit(circ_data)

Circuit(circuit_json='{"qubits":[{"id":0,"numResults":1},{"id":1,"numResults":1},{"id":2,"numResults":0},{"id"…

In [3]:
# Run without any noise
Histogram(qasm.run(code.ethylene, shots=1000))

Histogram(buckets={'[One, Zero]': 966, '[Zero, One]': 34}, shot_count=1000)

In [4]:
# Run with some Pauli (only) noise and qubit loss (both apply equally to all operations)
# This is built in the sparse simulator that also runs in VS Code today
results = qasm.run(code.ethylene, shots=1000, noise=BitFlipNoise(0.01), qubit_loss=0.01)
Histogram(results)

Histogram(buckets={'[One, Zero]': 720, '[Loss, Zero]': 73, '[Zero, One]': 48, '[Zero, Zero]': 58, '[Loss, One]…

In [ ]:
# Run with a noise model for different noise (including non-Pauli) and qubit loss per operation
# NOTE: 'nasm' is a separate package just this demo. It's just a few Python scripts we'd put in the existing QDK.

from nasm import Simulator, create_default_noise_model, amplitude_damping_kraus, to_results


# Default noise model predefines common gates but without any noise
noise = create_default_noise_model()

# Add amplitude damping noise to the noise model
noise.add_kraus_operator("amp_damp_015", amplitude_damping_kraus(0.15))
noise.update_gate_noise("sx", "amp_damp_015")
noise.update_gate_noise("rz", "amp_damp_015")

# Let's assume moving stuff into the zone for CZ gates is lossy too
noise.update_gate_loss("cz", 0.1)

# The noisy simulator runs on QIR, so compile to QIR first.
qir = qasm.compile(qasm_src)
sim = Simulator(noise_model=noise)
results = to_results(sim.run(qir, shots=1000))

Histogram(results)

Histogram(buckets={'[Loss, Loss]': 44, '[One, Zero]': 426, '[Loss, Zero]': 140, '[Zero, Zero]': 169, '[Zero, L…